<a href="https://colab.research.google.com/github/VladGeekPro/DeepLearning/blob/main/DeepLearningLab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Montarea Google Drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Citirea fișierului CSV (specificați calea corectă în Drive)
csv_path = "/content/drive/MyDrive/DeepLearningIASD2401/Videos/emisiuni.csv"
df = pd.read_csv(csv_path)

# (Optional) Afișăm primele rânduri pentru a verifica structura
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   NumeFisier
0  video1.mp4
1  video2.mp4
2  video3.mp4
3  video4.mp4
4  video5.mp4


In [ ]:
# Dacă CSV are o coloană 'NumeFisier' cu numele fișierelor video .mp4, construim calea completă
if 'NumeFisier' in df.columns:
    video_dir = "/content/drive/MyDrive/DeepLearningIASD2401/Videos/"  # directorul unde se află fișierele video
    df['video_path'] = video_dir + "/" + df['NumeFisier'].astype(str)
else:
    # Presupunem că există deja o coloană 'video_path' cu calea completă
    df['video_path'] = df['video_path'].astype(str)

print("Număr de emisiuni de procesat:", len(df))


Număr de emisiuni de procesat: 6


In [ ]:
# Instalăm ffmpeg (dacă e necesar) și librării auxiliare
!apt-get -qq install ffmpeg  # instalare silențioasă a ffmpeg
import os

# Creăm un folder pentru audio convertit (în Google Drive, lângă fișierele video)
audio_dir = "/content/drive/MyDrive/DeepLearningIASD2401/audio_wav"
os.makedirs(audio_dir, exist_ok=True)

# Parcurgem fiecare emisiune din DataFrame și extragem audio
audio_paths = []
for idx, row in df.iterrows():
    video_file = row['video_path']
    # Construim calea pentru fișierul WAV (același nume ca video, dar extensie .wav, salvat în audio_dir)
    base_name = os.path.splitext(os.path.basename(video_file))[0]
    wav_path = os.path.join(audio_dir, base_name + ".wav")
    audio_paths.append(wav_path)

    # Comanda ffmpeg de extragere audio mono 16kHz
    !ffmpeg -i "$video_file" -ac 1 -ar 16000 "$wav_path" -y

# Adăugăm calea audio în DataFrame
df['audio_path'] = audio_paths


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
print(df[['video_path','audio_path']].head())


                                          video_path  \
0  /content/drive/MyDrive/DeepLearningIASD2401/Vi...   
1  /content/drive/MyDrive/DeepLearningIASD2401/Vi...   
2  /content/drive/MyDrive/DeepLearningIASD2401/Vi...   
3  /content/drive/MyDrive/DeepLearningIASD2401/Vi...   
4  /content/drive/MyDrive/DeepLearningIASD2401/Vi...   

                                          audio_path  
0  /content/drive/MyDrive/DeepLearningIASD2401/au...  
1  /content/drive/MyDrive/DeepLearningIASD2401/au...  
2  /content/drive/MyDrive/DeepLearningIASD2401/au...  
3  /content/drive/MyDrive/DeepLearningIASD2401/au...  
4  /content/drive/MyDrive/DeepLearningIASD2401/au...  


In [ ]:
# !pip install -q transformers datasets torchaudio librosa
# from transformers import pipeline

# # Inițializăm pipeline-ul de ASR cu modelul pre-antrenat românesc
# asr_model_name = "gigant/romanian-wav2vec2"
# asr = pipeline("automatic-speech-recognition", model=asr_model_name, device=0)




In [ ]:
# import glob
# import os

# # Директория для сегментов
# os.makedirs("/content/chunks", exist_ok=True)

# # Словарь для хранения обучающих данных
# train_data = {"path": [], "text": []}

# # Проход по строкам DataFrame
# for idx, row in df.iterrows():
#     audio_file = row['audio_path']
#     base_name = os.path.splitext(os.path.basename(audio_file))[0]

#     # Распечатать имя текущего аудио
#     print(f"\n🔹 Обработка файла: {audio_file}")

#     # Сегментация аудио на куски по ~30 секунд
#     !ffmpeg -hide_banner -loglevel error -i "$audio_file" -f segment -segment_time 10 -c copy "/content/chunks/{base_name}_%03d.wav"

#     # Получение всех сегментов
#     segments = sorted(glob.glob(f"/content/chunks/{base_name}_*.wav"))
#     print(f"  ➤ Найдено сегментов: {len(segments)}")

#     full_transcript = ""

#     if len(segments) == 0:
#         print("  ⚠️ Нет сегментов. Пропускаем.")
#         df.at[idx, "Textul Recunoscut"] = ""
#         continue

#     for seg_path in segments:
#         print(f"    🎧 Сегмент: {seg_path}")
#         try:
#             result = asr(seg_path)

#             # Проверка ключа "text"
#             if isinstance(result, dict) and "text" in result:
#                 text_segment = result["text"]
#             else:
#                 print(f"    ⚠️ Ошибка ASR-результата: {result}")
#                 text_segment = ""
#         except Exception as e:
#             print(f"    ❌ Ошибка при ASR: {e}")
#             text_segment = ""

#         full_transcript += text_segment + " "

#         # Добавляем сегмент и текст в обучающий набор
#         train_data["path"].append(seg_path)
#         train_data["text"].append(text_segment)

#     # Удаление лишних пробелов
#     full_transcript = full_transcript.strip()

#     # Записываем результат в DataFrame
#     df.at[idx, "Textul Recunoscut"] = full_transcript
#     print(f"✅ Эмиссия {idx+1}/{len(df)} завершена. Длина текста: {len(full_transcript)} символов")


In [ ]:
# # Vizualizăm primele 200 de caractere din prima transcriere automată
# print("Exemplu transcriere automată:\n", df.at[0, "Textul Recunoscut"], "...")


In [ ]:
# Установка Whisper
!pip install -q openai-whisper ffmpeg-python

import whisper
import os
import glob
import pandas as pd

# Загрузка модели Whisper (можно: tiny, base, small, medium, large)
model = whisper.load_model("medium")  # "medium" точнее, но медленнее

# Убедимся, что директория под сегменты существует
os.makedirs("/content/chunks", exist_ok=True)

# Словарь для хранения обучающих данных
train_data = {"path": [], "text": []}

# Обработка каждого аудиофайла в DataFrame
for idx, row in df.iterrows():
    audio_file = row['audio_path']
    base_name = os.path.splitext(os.path.basename(audio_file))[0]

    print(f"\n🔹 Обработка файла: {audio_file}")

    # Разбиваем на 10-секундные куски с помощью ffmpeg
    os.system(f'ffmpeg -hide_banner -loglevel error -i "{audio_file}" -f segment -segment_time 10 -c copy "/content/chunks/{base_name}_%03d.wav"')

    # Список сегментов
    segments = sorted(glob.glob(f"/content/chunks/{base_name}_*.wav"))
    print(f"  ➤ Найдено сегментов: {len(segments)}")

    full_transcript = ""

    if len(segments) == 0:
        print("  ⚠️ Нет сегментов. Пропускаем.")
        df.at[idx, "Textul Recunoscut"] = ""
        continue

    for seg_path in segments:
        print(f"    🎧 Сегмент: {seg_path}")
        try:
            result = model.transcribe(seg_path, language="ro")  # "ro" = румынский
            text_segment = result["text"].strip()
        except Exception as e:
            print(f"    ❌ Ошибка при распознавании: {e}")
            text_segment = ""

        full_transcript += text_segment + " "
        train_data["path"].append(seg_path)
        train_data["text"].append(text_segment)

    # Запись транскрипции в DataFrame
    df.at[idx, "Textul Recunoscut"] = full_transcript.strip()
    print(f"✅ Эмиссия {idx+1}/{len(df)} завершена. Длина текста: {len(full_transcript)} символов")



🔹 Обработка файла: /content/drive/MyDrive/DeepLearningIASD2401/audio_wav/video1.wav
  ➤ Найдено сегментов: 212
    🎧 Сегмент: /content/chunks/video1_000.wav
    🎧 Сегмент: /content/chunks/video1_001.wav
    🎧 Сегмент: /content/chunks/video1_002.wav
    🎧 Сегмент: /content/chunks/video1_003.wav
    🎧 Сегмент: /content/chunks/video1_004.wav
    🎧 Сегмент: /content/chunks/video1_005.wav
    🎧 Сегмент: /content/chunks/video1_006.wav
    🎧 Сегмент: /content/chunks/video1_007.wav
    🎧 Сегмент: /content/chunks/video1_008.wav
    🎧 Сегмент: /content/chunks/video1_009.wav
    🎧 Сегмент: /content/chunks/video1_010.wav
    🎧 Сегмент: /content/chunks/video1_011.wav
    🎧 Сегмент: /content/chunks/video1_012.wav
    🎧 Сегмент: /content/chunks/video1_013.wav
    🎧 Сегмент: /content/chunks/video1_014.wav
    🎧 Сегмент: /content/chunks/video1_015.wav
    🎧 Сегмент: /content/chunks/video1_016.wav
    🎧 Сегмент: /content/chunks/video1_017.wav
    🎧 Сегмент: /content/chunks/video1_018.wav
    🎧 Сегмент:

In [ ]:
# Vizualizăm primele 200 de caractere din prima transcriere automată
print("Exemplu transcriere automată:\n", df.at[0, "Textul Recunoscut"], "...")

Exemplu transcriere automată:
 Vă mulțumesc frumos pentru vizionare! Nu o vom abandona, dar există și o știeră foarte caldă, să spunem așa, proaspătă de tot, care a strinit deja nenumărată reacții și suntem cumva datori să o abordăm. Și anume aceea că practic, Marea... Veste este faptul că domnul Octavianței Cu a hotărât să părăsească Pepe de Asta Să părăsească cumva și statutul pe care l-a avut acela de deputat al blocului acum. Rămână deputat independent în cadrul blocului acum, pe de o parte și pe de altă parte adresându-să. atât León Ceban că ceilui Andrei Năstase, își lansează și candidatura pentru Primoria Capitalei. O mișcare care a stârnit deja înănumărate reacții, o mișcare... care nu rămâne fără consecință și fără urmări. Și e interesant să vedem cum ne putem raporta la această mișcare. Invitații din această seară erau pregătiți pentru un alt subuniment. un subiect care se referă la felul în care Canal 2 a anunțat că renunță la frecvența națională pe care o deținea în urma pr

In [ ]:
# # Salvăm transcrierile inițiale într-un CSV separat (în Drive) pentru verificare externă
# df.to_csv("/content/drive/MyDrive/DeepLearningIASD2401/Videos/emisiuni_transcrieri_initiale.csv", index=False)
# print("Transcrierile inițiale au fost salvate. Puteți verifica 'emisiuni_transcrieri_initiale.csv' în Drive.")


In [ ]:
from datasets import Dataset, Audio

# Construim Dataset-ul din dicționarul de date
dataset = Dataset.from_dict(train_data)
# (Optional) amestecăm exemplele pentru a distribui uniform
dataset = dataset.shuffle(seed=42)

# Împărțim datele: 90% antrenament, 10% validare
dataset_split = dataset.train_test_split(test_size=0.1, seed=42)
train_ds = dataset_split["train"]
eval_ds = dataset_split["test"]

# Convertim coloana "path" (către fișier) într-o coloană audio reală
train_ds = train_ds.cast_column("path", Audio(sampling_rate=16000))
eval_ds = eval_ds.cast_column("path", Audio(sampling_rate=16000))

print(f"Exemple pentru antrenament: {len(train_ds)}, pentru validare: {len(eval_ds)}")


Exemple pentru antrenament: 1107, pentru validare: 124


In [ ]:
import json

# Construim setul de caractere folosite în transcrieri
all_texts = " ".join(train_data["text"])
vocab_chars = sorted(list(set(all_texts.replace(" ", ""))))  # toate caracterele, fără spații
# Adăugăm caracterul '|' ca reprezentare a spațiului
vocab_chars.append("|")
# Construim dicționarul de vocab (caracter -> index)
vocab_dict = {ch: i for i, ch in enumerate(vocab_chars)}
# Adăugăm tokenii speciali necesari
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

# Salvăm vocabularul într-un fișier json
with open("vocab.json", "w") as f:
    json.dump(vocab_dict, f)

print("Dimensiune vocabular:", len(vocab_dict))
print("Exemplu mapare:", list(vocab_dict.items())[:10])


Dimensiune vocabular: 86
Exemplu mapare: [('!', 0), ('%', 1), ("'", 2), (',', 3), ('-', 4), ('.', 5), ('0', 6), ('1', 7), ('2', 8), ('3', 9)]


In [ ]:
list(vocab_dict.items())

[('!', 0),
 ('%', 1),
 ("'", 2),
 (',', 3),
 ('-', 4),
 ('.', 5),
 ('0', 6),
 ('1', 7),
 ('2', 8),
 ('3', 9),
 ('4', 10),
 ('5', 11),
 ('6', 12),
 ('7', 13),
 ('8', 14),
 ('9', 15),
 ('<', 16),
 ('>', 17),
 ('?', 18),
 ('A', 19),
 ('B', 20),
 ('C', 21),
 ('D', 22),
 ('E', 23),
 ('F', 24),
 ('G', 25),
 ('H', 26),
 ('I', 27),
 ('J', 28),
 ('K', 29),
 ('L', 30),
 ('M', 31),
 ('N', 32),
 ('O', 33),
 ('P', 34),
 ('R', 35),
 ('S', 36),
 ('T', 37),
 ('U', 38),
 ('V', 39),
 ('W', 40),
 ('Y', 41),
 ('Z', 42),
 ('a', 43),
 ('b', 44),
 ('c', 45),
 ('d', 46),
 ('e', 47),
 ('f', 48),
 ('g', 49),
 ('h', 50),
 ('i', 51),
 ('j', 52),
 ('k', 53),
 ('l', 54),
 ('m', 55),
 ('n', 56),
 ('o', 57),
 ('p', 58),
 ('q', 59),
 ('r', 60),
 ('s', 61),
 ('t', 62),
 ('u', 63),
 ('v', 64),
 ('w', 65),
 ('x', 66),
 ('y', 67),
 ('z', 68),
 ('|', 84),
 ('Î', 70),
 ('à', 71),
 ('â', 72),
 ('î', 73),
 ('ó', 74),
 ('ă', 75),
 ('ć', 76),
 ('Ş', 77),
 ('ş', 78),
 ('ţ', 79),
 ('Ș', 80),
 ('ș', 81),
 ('Ț', 82),
 ('ț', 83),
 (

In [ ]:
!pip install -U transformers datasets evaluate jiwer
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor

# Inițializăm tokenizer-ul CTC cu vocabularul nostru
tokenizer = Wav2Vec2CTCTokenizer("vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
# Inițializăm feature extractor-ul audio (normalizare mean-std)
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, do_normalize=True, return_attention_mask=True)
# Combinăm într-un procesor unic
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [ ]:
from transformers import Wav2Vec2ForCTC

# model_name = "facebook/wav2vec2-large-xlsr-53"
model_name = "gigant/romanian-wav2vec2"

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    ctc_loss_reduction="mean",
    pad_token_id=tokenizer.pad_token_id,
    vocab_size=len(tokenizer)
)

# Замораживаем нижние слои
model.freeze_feature_encoder()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from dataclasses import dataclass

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: bool = True

    def __call__(self, features):
        # Separăm intrările de label-uri
        input_features = [{"input_values": f["input_values"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]
        # Aplicăm padding pe input_values
        batch = self.processor.feature_extractor.pad(input_features, padding=self.padding, return_tensors="pt")
        # Aplicăm padding pe labels
        with self.processor.as_target_processor():
            labels_batch = self.processor.tokenizer.pad(label_features, padding=self.padding, return_tensors="pt")
            labels = labels_batch["input_ids"]
            # Înlocuim pad_token_id cu -100 pentru a fi ignorat la calculul pierderii CTC
            labels[labels == self.processor.tokenizer.pad_token_id] = -100
            batch["labels"] = labels
        return batch

# Inițializăm colatorul cu procesorul nostru
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [ ]:
!pip install -q jiwer
!pip install -q evaluate
import torch
import evaluate

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_logits = torch.tensor(pred.predictions)
    pred_ids = torch.argmax(pred_logits, dim=-1)
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)

    label_ids = torch.tensor(pred.label_ids)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # Удаляем пустые строки из обоих списков
    clean_preds = []
    clean_refs = []
    for p, r in zip(pred_str, label_str):
        if r.strip():  # если строка не пустая
            clean_preds.append(p)
            clean_refs.append(r)

    if len(clean_refs) == 0:
        return {"wer": None}

    wer_val = wer_metric.compute(predictions=clean_preds, references=clean_refs)
    return {"wer": wer_val}



In [ ]:
# Создаем копии перед обработкой
train_ds_2 = train_ds
eval_ds_2 = eval_ds

# Функция для переименования и обработки
def rename_and_process(batch):
    audio_array = batch["path"]["array"]
    sampling_rate = batch["path"]["sampling_rate"]

    # Обработка аудиосигнала
    batch["input_values"] = processor(audio_array, sampling_rate=sampling_rate).input_values[0]

    # Обработка текста в токены
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids

    return batch

# Применяем преобразование
train_ds = train_ds.map(rename_and_process, remove_columns=train_ds.column_names)
eval_ds = eval_ds.map(rename_and_process, remove_columns=eval_ds.column_names)


Map:   0%|          | 0/1107 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/124 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

output_dir = "/content/drive/MyDrive/DeepLearningIASD2401/ASR_model"  # director in Drive pentru a salva checkpont-uri și modelul final

training_args = TrainingArguments(
    output_dir=output_dir,
    group_by_length=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # 8*2 = 16 efectiv
    eval_strategy="epoch",
    num_train_epochs=5,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=1e-4,
    warmup_steps=500,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,  # pentru ca Trainer să știe cum să facă padding la intrare
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-33-f52a872ff406>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


Sfat: Dacă observați că WER-ul de validare nu mai scade sau chiar crește, modelul ar putea supra-antrena. Vă puteți opri mai devreme (reduceți num_train_epochs sau folosiți early_stopping dacă implementați). De asemenea, puteți să debLocați feature extractor-ul după 1-2 epoci (apelând model.freeze_feature_extractor() inițial, apoi model.unfreeze_feature_extractor() și continuând antrenarea) pentru a îmbunătăți și mai mult performanța – dar atenție la posibilul overfitting.

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Wer
1,No log,inf,1.000000
2,36.723400,inf,1.000000


In [ ]:
# Salvează manual modelul final și procesorul (deși Trainer ar fi salvat ultimul checkpoint oricum)
trainer.save_model(output_dir)  # salvează modelul (pytorch_model.bin + config)
processor.save_pretrained(output_dir)  # salvează tokenizerul și feature extractor-ul
print("Modelul fine-tunat a fost salvat în:", output_dir)


## Dupa salvarea modelului antrenat și testarea lui pe un fișier nou
Pentru a evalua performanța modelului fine-tunat, îl vom testa pe un fișier audio nou.
### Vom parcurge pașii:
- Extragerea audio din fișierul video nou (similar ca înainte, folosind ffmpeg).
- Folosirea modelul antrenat pentru a transcrie audio-ul.
- Compararea transcrierei cu așteptările sau calcularea WER dacă avem transcriere de referință.

In [ ]:
# 1. Extragerea audio din noul fișier video
test_video_path = "/content/drive/MyDrive/DeepLearningIASD2401/Videos/test.mp4"  # <- înlocuiți cu calea reală

!ffmpeg -hide_banner -loglevel error -i "$test_video_path" -ac 1 -ar 16000 "/content/test.wav" -y

# 2. Încărcarea modelului antrenat
from transformers import AutoProcessor, AutoModelForCTC
proc = AutoProcessor.from_pretrained(output_dir)  # încarcă procesorul salvat
model_ft = AutoModelForCTC.from_pretrained(output_dir)
model_ft.to("cuda")  # mutăm modelul pe GPU pentru viteză

# 3. Transcrierea audio-ului de test
import librosa
speech, sr = librosa.load("/content/test.wav", sr=16000)
input_vals = proc(speech, sampling_rate=16000, return_tensors="pt", padding=True)
input_vals = {k: v.to("cuda") for k,v in input_vals.items()}
with torch.no_grad():
    logits = model_ft(**input_vals).logits
pred_ids = torch.argmax(logits, dim=-1)
transcriere = proc.batch_decode(pred_ids)[0]
print("Transcriere automată (model fine-tunat):\n", transcriere)
